In [1]:
import argparse
import os
import pathlib
import sys
import time

import numpy as np
import pandas as pd
import psutil
import skimage

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import gc

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

sys.path.append(f"{root_dir}/3.cellprofiling/featurization_utils/")
from area_size_shape_utils import measure_3D_area_size_shape
from area_size_shape_utils_gpu import measure_3D_area_size_shape_gpu
from featurization_parsable_arguments import parse_featurization_args
from loading_classes import ImageSetLoader, ObjectLoader
from resource_profiling_util import get_mem_and_time_profiling

In [2]:
if not in_notebook:
    arguments_dict = parse_featurization_args()
    patient = arguments_dict["patient"]
    well_fov = arguments_dict["well_fov"]
    compartment = arguments_dict["compartment"]
    channel = arguments_dict["channel"]
    processor_type = arguments_dict["processor_type"]

else:
    well_fov = "C4-2"
    patient = "NF0014"
    compartment = "Nuclei"
    channel = "DNA"
    processor_type = "GPU"

image_set_path = pathlib.Path(
    f"{root_dir}/data/{patient}/profiling_input_images/{well_fov}/"
)

output_parent_path = pathlib.Path(
    f"{root_dir}/data/{patient}/extracted_features/{well_fov}/"
)
output_parent_path.mkdir(parents=True, exist_ok=True)

In [3]:
channel_n_compartment_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
start_time = time.time()
# get starting memory (cpu)
start_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2

In [5]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    anisotropy_spacing=(1, 0.1, 0.1),
    channel_mapping=channel_n_compartment_mapping,
)

In [6]:
object_loader = ObjectLoader(
    image_set_loader.image_set_dict["DNA"],
    image_set_loader.image_set_dict[compartment],
    "DNA",
    compartment,
)


# area, size, shape
if processor_type == "GPU":
    size_shape_dict = measure_3D_area_size_shape_gpu(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
elif processor_type == "CPU":
    size_shape_dict = measure_3D_area_size_shape(
        image_set_loader=image_set_loader,
        object_loader=object_loader,
    )
else:
    raise ValueError(
        f"Processor type {processor_type} is not supported. Use 'CPU' or 'GPU'."
    )

In [7]:
final_df = pd.DataFrame(size_shape_dict)

# prepend compartment and channel to column names
for col in final_df.columns:
    if col not in ["object_id"]:
        final_df.rename(
            columns={col: f"Area.Size.Shape_{compartment}_{col}"},
            inplace=True,
        )
final_df.insert(1, "image_set", image_set_loader.image_set_name)

output_file = pathlib.Path(
    output_parent_path
    / f"AreaSize_Shape_{compartment}_{processor_type}_features.parquet"
)
final_df.to_parquet(output_file)
final_df.head()

object_id image_set  Area.Size.Shape_Nuclei_VOLUME  \
0          6      C4-2                        12101.0   
1         11      C4-2                       223428.0   
2         21      C4-2                       157264.0   
3         25      C4-2                       191095.0   
4         49      C4-2                       186447.0   

   Area.Size.Shape_Nuclei_CENTER.X  Area.Size.Shape_Nuclei_CENTER.Y  \
0                       177.255351                       363.403686   
1                       747.947889                       669.849607   
2                       612.898311                       288.309670   
3                       485.697794                       479.110327   
4                       598.913166                       579.334309   

   Area.Size.Shape_Nuclei_CENTER.Z  Area.Size.Shape_Nuclei_BBOX.VOLUME  \
0                         3.525742                             16830.0   
1                        13.842728                           8905680.0   
2                        10.146613                           1372161.0   
3                        14.099982                            564200.0   
4                         8.307170                           2901600.0   

   Area.Size.Shape_Nuclei_MIN.X  Area.Size.Shape_Nuclei_MAX.X  \
0                           148                           203   
1                           501                           879   
2                           453                           814   
3                           410                           565   
4                           421                           811   

   Area.Size.Shape_Nuclei_MIN.Y  Area.Size.Shape_Nuclei_MAX.Y  \
0                           339                           390   
1                           165                           925   
2                           195                           376   
3                           411                           541   
4                           343                           808   

   Area.Size.Shape_Nuclei_MIN.Z  Area.Size.Shape_Nuclei_MAX.Z  \
0                             1                             7   
1                             1                            32   
2                             0                            21   
3                             0                            28   
4                             1                            17   

   Area.Size.Shape_Nuclei_EXTENT  Area.Size.Shape_Nuclei_EULER.NUMBER  \
0                       0.719014                                    1   
1                       0.025088                                    4   
2                       0.114610                                    2   
3                       0.338701                                    2   
4                       0.064257                                    2   

   Area.Size.Shape_Nuclei_EQUIVALENT.DIAMETER  \
0                                   28.484431   
1                                   75.285787   
2                                   66.969476   
3                                   71.463382   
4                                   70.879219   

   Area.Size.Shape_Nuclei_SURFACE.AREA  
0                            74.645814  
1                         14086.904413  
2                          2465.269977  
3                          1404.419471  
4                          5562.146207

In [10]:
end_mem = psutil.Process(os.getpid()).memory_info().rss / 1024**2
end_time = time.time()
get_mem_and_time_profiling(
    start_mem=start_mem,
    end_mem=end_mem,
    start_time=start_time,
    end_time=end_time,
    feature_type="AreaSizeShape",
    well_fov=well_fov,
    patient_id=patient,
    channel="DNA",
    compartment=compartment,
    CPU_GPU=processor_type,
    output_file_dir=pathlib.Path(
        f"{root_dir}/data/{patient}/extracted_features/run_stats/{well_fov}_AreaSizeShape_DNA_{compartment}_{processor_type}.parquet"
    ),
)


        Memory and time profiling for the run:

        Patient ID: NF0014

        Well and FOV: C4-2

        Feature type: AreaSizeShape

        CPU/GPU: GPU")

        Memory usage: 1623.52 MB

        Time:

        --- %s seconds --- % 27.086897611618042

        --- %s minutes --- % 0.45144829352696736

        --- %s hours --- % 0.0075241382254494565
    


True